# 査読者に公開した論拠データを更新する
査読者に公開した論拠データを最新化するタスクです。<br>
リプライを対応の中で論拠データを編集した場合のみ、実行してください。

## リプライのためのデータを再配置する
査読者に公開している論拠データを最新のものに更新してください。<br>
査読用プロジェクトを選択し、「プロジェクトを表示する」ボタンを押下することでGakuNin RDMのプロジェクトページが表示されされます。<br>
そこで公開している論拠データを最新のものへと更新してください。

In [ ]:
# プロジェクトを表示する
import os

import panel as pn

from library.task_director import TaskDirector

from IPython.display import display
from IPython.core.display import Javascript
from library.utils.config import connect as con_config, message as msg_config
from library.utils.html.button import create_button
from library.utils.input import get_project_id
from library.utils.storage_provider import grdm

notebook_name = 'publish_argument_data_to_reviewers.ipynb'
button_width = 500

class Publish(TaskDirector):

    def __init__(self, working_path: str) -> None:

        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def generate_button(self):

        project_id = get_project_id()

        obj = create_button(
            url=self.grdm.build_main_menu_url(self.grdm_url, project_id),
            msg=msg_config.get('publish_argument_data_to_reviewers', 'go_grdm'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        pn.extension()
        display(pn.pane.HTML(obj, width=button_width))
        display(Javascript('IPython.notebook.save_checkpoint();'))

Publish(os.path.abspath('__file__')).generate_button()

In [ ]:
# 論拠データフォルダを表示する
import os
import panel as pn
from IPython.display import display
from IPython.core.display import Javascript
from library.utils.access import open_data_folder

folder_name = 'argument_data'
button_width = 500

button = open_data_folder(os.path.abspath('__file__'))

pn.extension()
display(pn.pane.HTML(button, width=button_width))
display(Javascript('IPython.notebook.save_checkpoint();'))

## 再現性を確認する
査読者に公開した論拠データが再現可能な状態であることを確認してください。<br>
再現性を確認する機能を現在開発中のため、再現性の確認は手動で行ってください。

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()